In [1]:
using LinearAlgebra
using Distributions
using Optim
using Random
using StatsFuns
using JuMP
using MosekTools
using StatsBase
using SparseArrays # 可选，用于处理稀疏性（如果数据量很大）
using FileIO
using JLD2
using Plots
using LaTeXStrings
using DataFrames, Colors
using StatsPlots   # 提供 boxplot，基于 Plots

include("Params_PLD.jl")
include("Data_Generation_PLD.jl")
include("Estimation_PLD.jl")
include("Estimation_PLD_Fast.jl")
include("Models_PLD.jl")
include("Evaluation_PLD.jl")
include("Implement_All_Methods_PLD.jl")
include("Figures_PLD.jl")

hist_profit_distribution (generic function with 1 method)

In [2]:
Params = get_default_params_PLD()
N = Params["N"] # number of products
N_x = Params["N_x"] # dimension of product features
c_l = Params["c_l"] 
d_r = Params["d_r"]
rev_gap = Params["rev_gap"]
N_u = Params["N_u"] # dimension of customer features
S_train_all = Params["S_train_all"] # training data size
S_test = Params["S_test"] # test data size
N_Max = Params["N_Max"] # maximum assortment size
N_nonzero = Params["N_nonzero"] # number of nonzero entries in A
Time_Limit = Params["Time_Limit"] # time limit for optimization
dual_norm = Params["dual_norm"] # dual norm for robust optimization
gamma_list = Params["gamma_list"] # list of gamma values for robust optimization
psi_lb = Params["psi_lb"] # lower bound for psi
psi_ub = Params["psi_ub"] # upper bound for psi
phi_lb = Params["phi_lb"]   # lower bound for phi
phi_ub = Params["phi_ub"]  # upper bound for phi
num_c = Params["num_c"] # number of customer segments
instances = Params["instances"] # number of instances
seed = Params["seed"] # random seed
coef_this = Params["coef_this"] # coefficient for data generation
coef_Wang_Qi_Shen = Params["coef_Wang_Qi_Shen"] # coefficient for Wang, Qi, Shen data generation

Random.seed!(seed)

TaskLocalRNG()

In [3]:
function compute_w(params,z_input)
    alpha0 = params.alpha0
    alpha = params.alpha
    beta = params.beta
    A = params.A
    nu0 = alpha0 + beta' * z_input;
    nu = alpha .+ A * z_input;
    return nu0,nu
end

compute_w (generic function with 1 method)

In [4]:
N_x = 5
N_u = 1
N_nonzero = 1
theta_true_Fixed, r_params_Fixed = Generate_Wang_Qi_Max_True_Paras(N_x,N_u,N_nonzero,coef_Wang_Qi_Shen);
S_train = 10000
is_ridge = false
lbd = 0.0

0.0

In [5]:
Input_Data_this = Generate_Data_this_Same_Para(N_Max,N_x,N_u,S_train,S_test,theta_true_Fixed, r_params_Fixed);
theta_true,r_params,X_train,Y_train,Z_train,asorrtment_train,X_test,Y_test,Z_test = Get_Input_Data(Input_Data_this);

In [6]:
alp_0_exp, alpha_exp, beta_exp, A_exp, solve_time, sol_status, obj_val = Estimate_OPT_Model_PLD(N_Max,N_x,N_u,Y_train,X_train,Z_train, asorrtment_train,is_ridge, lbd);

In [7]:
theta_Exp = (alpha0 = alp_0_exp,
              alpha = alpha_exp,
              beta = beta_exp,
              A = A_exp)
nu0_Exp,nu_Exp = compute_w(theta_Exp,Z_test[1,:]);
nu0_true,nu_true = compute_w(theta_true,Z_test[1,:]);

# theta_hat = Estimation_This(0.0,X_train,Y_train,Z_train,N_x,N_u)
alpha0_hat, alpha_hat, beta_hat, A_hat, opt_result, X_tilde = estimate_parameters_fast(X_train, Y_train, Z_train, 0.0, N_x, N_u);
theta_hat = (alpha0=alpha0_hat, alpha=alpha_hat, beta=beta_hat, A=A_hat)
nu0_hat,nu_hat = compute_w(theta_hat,Z_test[1,:])  

println("nu0 true = ",round(nu0_true,digits=4),", nu_true = ", round.(nu_true,digits=4))
println("nu0 hat = ",round(nu0_hat,digits=4),", nu_hat = ", round.(nu_hat,digits=4))
println("nu0 Exp = ",round(nu0_Exp,digits=4),", nu_Exp = ", round.(nu_Exp,digits=4))

  0.001888 seconds (20.00 k allocations: 5.740 MiB)
nu0 true = 1.0069, nu_true = [0.3615, 0.4707, 0.6213, -0.5172, 0.7339]
nu0 hat = 1.0389, nu_hat = [0.3636, 0.4557, 0.6638, -0.5077, 0.7549]
nu0 Exp = 1.0389, nu_Exp = [0.3636, 0.4557, 0.6638, -0.5077, 0.7549]


In [9]:
S_train = 10000
for s in 1:5
    println("----- Trial $(s) -----")
    theta_true_Fixed, r_params_Fixed = Generate_Wang_Qi_Max_True_Paras(N_x,N_u,N_nonzero,coef_Wang_Qi_Shen);
    Input_Data_this = Generate_Data_this_Same_Para(N_Max,N_x,N_u,S_train,S_test,theta_true_Fixed, r_params_Fixed);
    theta_true,r_params,X_train,Y_train,Z_train,asorrtment_train,X_test,Y_test,Z_test = Get_Input_Data(Input_Data_this);
    is_ridge = false
    lbd = 0.0
    alp_0_exp, alpha_exp, beta_exp, A_exp, solve_time, sol_status, obj_val = Estimate_OPT_Model_PLD(N_Max,N_x,N_u,Y_train,X_train,Z_train, asorrtment_train,is_ridge, lbd);

    theta_Exp = (alpha0 = alp_0_exp,
              alpha = alpha_exp,
              beta = beta_exp,
              A = A_exp)
    nu0_Exp,nu_Exp = compute_w(theta_Exp,Z_test[1,:]);
    nu0_true,nu_true = compute_w(theta_true,Z_test[1,:]);

    # # theta_hat = Estimation_This(0.0,X_train,Y_train,Z_train,N_x,N_u)
    # alpha0_hat, alpha_hat, beta_hat, A_hat, opt_result, X_tilde = estimate_parameters_fast(X_train, Y_train, Z_train, 0.0, N_x, N_u);
    # theta_hat = (alpha0=alpha0_hat, alpha=alpha_hat, beta=beta_hat, A=A_hat)
    # nu0_hat,nu_hat = compute_w(theta_hat,Z_test[1,:])  

    println("nu0 true = ",round(nu0_true,digits=4),", nu_true = ", round.(nu_true,digits=4))
    # println("nu0 hat = ",round(nu0_hat,digits=4),", nu_hat = ", round.(nu_hat,digits=4))
    println("nu0 Exp = ",round(nu0_Exp,digits=4),", nu_Exp = ", round.(nu_Exp,digits=4))
end

----- Trial 1 -----
nu0 true = 1.3008, nu_true = [-0.8406, 0.3139, 0.1177, 0.0869, -0.2516]
nu0 Exp = 1.4898, nu_Exp = [-0.8103, 0.3028, 0.1048, 0.095, -0.2732]
----- Trial 2 -----
nu0 true = 1.8465, nu_true = [0.2826, -0.8702, -0.9513, 0.2465, -0.2794]
nu0 Exp = 1.8816, nu_Exp = [0.2639, -0.8368, -0.9303, 0.2794, -0.2806]
----- Trial 3 -----
nu0 true = 1.8483, nu_true = [0.9215, 0.8243, -0.1913, 0.3286, 0.4631]
nu0 Exp = 1.7834, nu_Exp = [0.9238, 0.8485, -0.1944, 0.3318, 0.4725]
----- Trial 4 -----
nu0 true = 1.2323, nu_true = [0.2983, 0.723, -0.5259, -0.6956, 0.3519]
nu0 Exp = 1.1001, nu_Exp = [0.2777, 0.738, -0.5193, -0.6527, 0.3655]
----- Trial 5 -----
nu0 true = 1.1445, nu_true = [-0.34, -0.529, -0.243, -0.1347, -0.4008]
nu0 Exp = 1.1677, nu_Exp = [-0.3863, -0.5012, -0.2706, -0.1757, -0.4168]
